# T1 and T2 measurements

In [7]:
from collections import OrderedDict
import datetime
import numpy as np
import time
import os

In [8]:
try: 
    pulsedmasterlogic
except NameError:
    manager.startModule('logic', 'pulsedmasterlogic')
try: 
    scannerlogic
except NameError:
    manager.startModule('logic', 'scannerlogic')
try: 
    optimizerlogic
except NameError:
    manager.startModule('logic', 'optimizerlogic')
try:
    sequencegeneratorlogic
except NameError:
    manager.startModule('logic', 'sequencegeneratorlogic')
try:
    pulsedmeasurement
except NameError:
    manager.startModule('gui', 'pulsedmeasurement')   
try:
    poimanagerlogic
except NameError:
    manager.startModule('logic', 'poimaganerlogic')    

In [9]:
def write_to_logfile(nametag, timestamp, name, **kwargs):
    """ Write parameters to custom logfile with name nametag """
    if type(timestamp) is not str:
        timestamp = str(timestamp)
    parameters = list(kwargs)
    if len(parameters) == 1 and type(kwargs[parameters[0]]) is OrderedDict:
        param_dict = kwargs[parameters[0]]
        parameters = list(param_dict)
        kwargs = param_dict
    log_dir = pulsedmeasurement._save_logic.get_path_for_module('CustomLogfiles')
    log_path = os.path.join(log_dir, nametag + '.txt')
    if not os.path.isfile(log_path):
        with open(log_path, 'w') as logfile:
            logfile.write('# timestamp\t')
            logfile.write('# Name\t')
            for param in parameters:
                logfile.write(param + '\t')
            logfile.write('\n#\n')
    with open(log_path, 'a') as logfile:
        logfile.write(timestamp + '\t')
        logfile.write(name + '\t')
        for param in parameters:
            logfile.write('{0:3.6e}\t'.format(kwargs[param]))
        logfile.write('\n')
    return

In [48]:
# static hardware parameters:
setup = OrderedDict()
setup['sampling_freq'] = pulsedmeasurementlogic.sample_rate
setup['fc_binwidth'] = pulsedmeasurementlogic.fast_counter_binwidth
setup['wait_length'] = 1.5e-6
setup['aom_delay'] = 500e-9
#setup['channel_config_name'] = pulsedmeasurement.current_channel_config_name
setup['seq_trig']=''
setup['gate']='d_ch2'
setup['NV_name']='NV1'
setup['poi'] = poimanagerlogic.active_poi.get_key()
setup['laser_length'] = 3.0e-6
setup["min_counts"] = 10000 # if after a refocus only 40kcounts are measured the refocus is repeated up to max 3 times
# Set amplitude in logic and pulser hardware:
amp_dict = OrderedDict()
amp_dict['a_ch1'] = 0.25
amp_dict['a_ch2'] = 0.0
AWG5014C.amplitude_dict = amp_dict
#pulsedmaster.pulse_generator_settings_changed(setup['sampling_freq'], setup['channel_config_name'], amp_dict, False)

In [5]:
# for T2:
T2 = OrderedDict()
T2['tau_start'] = 300e-9
T2['tau_step'] = 3.0e-6
T2['number_of_taus'] = 20 
T2['measurement_time'] = 10*60
T2['refocus_interval'] = 2*60
T2['alternating']=True
T2['rabi_period'] = 167.33*1e-9

In [12]:
do_hahn_echo_refocus(setup['poi'])

165936.333333 2.3914814909869716e-09
165936.333333 6.147379722744263e-09
165936.333333 1.8174969388247116e-09
165936.333333 7.808394206611571e-09


False

In [ ]:
mw_freq = 1392.82e+6 # in MHz
mw_power = -15

pulsedmeasurement._pa.ext_control_mw_freq_DoubleSpinBox.setValue(mw_freq-0.1e+9)
pulsedmeasurement._pa.ext_control_mw_power_DoubleSpinBox.setValue(mw_power)
pulsedmeasurement.ext_mw_params_changed()

In [47]:
# for T1:
T1 = OrderedDict()
T1['tau_start'] = 300e-9
T1['tau_step'] = 200.0e-6
T1['number_of_taus'] = 10
T1['measurement_time'] = 60*60
T1['refocus_interval'] = 2*60
T1['alternating']=True
T1['rabi_period'] = 173.0*1e-9

In [46]:
# generate new_T1 sequence
sequencegeneratorlogic.delete_sequence('T1 sequence')    
sequencegeneratorlogic.generate_T1_sequence('T1 sequence', T1['rabi_period'], 100.0e6, 0.25,
                              T1['tau_start'], T1['tau_step'], T1['number_of_taus'], 'a_ch1',
                                                      3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2')
# sample, upload and load waveform
if 'T1 sequence' not in sequencegeneratorlogic.saved_pulse_sequences:
    sequencegeneratorlogic.log.error('Sequence generation of T1 in notebook timed out.')
pulsedmasterlogic.sample_sequence('T1 sequence', True)


while pulsedmasterlogic.status_dict['sauplo_sequence_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(5)

import winsound
frequency = 300  # Set Frequency To 2500 Hertz
duration = 1000  # Set Duration To 1000 ms == 1 second
winsound.Beep(frequency, duration)

In [54]:
setup['poi'] = poimanagerlogic.active_poi.get_key()
refocus_poi(setup['poi'])

29124.0 1.7392439371001522e-09


True

In [58]:
do_T1_refocus(setup['poi'])

29124.0 9.272207817033305e-09
29124.0 3.5635772603836112e-09
29124.0 3.797019229064133e-09
29124.0 4.417814805070876e-09
29124.0 4.4173041216360456e-09
29124.0 5.341069969305945e-09
29124.0 4.476769684328536e-10
29124.0 1.784154823785745e-09
29124.0 5.729158242110712e-10
29124.0 2.5143462513719855e-09
29124.0 1.9521561158934036e-09
29124.0 8.629492163741688e-10
29124.0 6.37902551317755e-11
29124.0 3.1447786011084005e-10
29124.0 7.294970870779606e-09
29124.0 1.2406253904749564e-09
29124.0 3.3249238150264543e-09
29124.0 8.63452782333445e-10
29124.0 2.1692615351714484e-09
29124.0 7.745287309590969e-09
29124.0 3.2941768161938337e-10
29124.0 6.982966114198186e-09
2D gaussian fit not successfull
29124.0 0.0
29124.0 8.014557877795427e-09
29124.0 2.356094224041427e-09
29124.0 1.045665251604808e-10
29124.0 5.0619553955232174e-11
29124.0 2.7103782892847774e-09
29124.0 2.506374767225519e-09


Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 737, in start_pulsed_measurement
    self.pulse_generator_on()
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 451, in pulse_generator_on
    err = self._pulse_generator_device.pulser_on()
  File "C:\Users\pi3\Documents\GitHub\qudi\hardware\awg\tektronix_awg5014c.py", line 257, in pulser_on
    return self.get_status()[0]
TypeError: 'int' object is not subscriptable
Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 879, in stop_pulsed_measurement
    self.pulse_generator_off()
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\pulsed_measurement_logic.py", line 457, in pulse_generator_off
    err = self._pulse_generator_device.pulser_off()
  File "C:\Users\pi3\Documents\GitHub\qudi\hardware\awg\tektronix_awg5014c.py", line 268, in pulser_off
    return self.get_status

In [56]:
def refocus_position():
    # switch on laser
    pulsedmasterlogic.load_asset_into_channels('Laser_On')
    while pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmaster.toggle_pulse_generator(True)
    # perform refocus
    scanner.stop_scanning()
    crosshair_pos = scanner.get_position()
    optimizer.start_refocus(initial_pos=crosshair_pos)
    while optimizer.getState() == 'idle':
        time.sleep(0.2)
    while optimizer.getState() != 'idle':
        time.sleep(0.2)
    scanner.set_position('optimizer', x=optimizer.optim_pos_x, y=optimizer.optim_pos_y, z=optimizer.optim_pos_z, a=0.0)
    time.sleep(1)
    # switch off laser
    pulsedmaster.toggle_pulse_generator(False)
    
    # write new position to log file return
    timestamp = str(datetime.datetime.now())
    new_x = optimizer.optim_pos_x
    new_y = optimizer.optim_pos_y
    new_z = optimizer.optim_pos_z
#    drift = np.sqrt((new_x-position['x'])**2 + (new_y-position['y'])**2 + (new_z-position['z'])**2)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=new_x, y_pos=new_y, z_pos=new_z)
    return new_x, new_y, new_z

def refocus_poi(poi=None, max_drift=0.3,min_counts=setup["min_counts"]):
    if poi==None:
        return False
#     pulsedmasterlogic.load_asset_into_channels('Laser_On')
#     while pulsedmasterlogic.status_dict['loading_busy']:
#         time.sleep(0.5)
#     pulsedmasterlogic.toggle_pulse_generator(False)
#     time.sleep(0.5)
    # perform refocus
    scannerlogic.stop_scanning()
    poimanagerlogic.go_to_poi(poi)
    pos_start = scannerlogic.get_position()
    setup['NV_name']=poimanagerlogic.poi_list[poi]._name
    poimanagerlogic.optimise_poi(poi)
    while optimizerlogic.getState() == 'idle':
        time.sleep(0.2)
    while optimizerlogic.getState() != 'idle':
        time.sleep(0.2)
    time.sleep(5)
    pos_end=scannerlogic.get_position()
    print(counterlogic.countdata.mean(),abs(pos_end[0]-pos_start[0]))
    if (abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts):
        scannerlogic.set_position('',x=pos_start[0],y=pos_start[1],z=pos_start[2])
        poimanagerlogic.set_new_position(poi,pos_start)
        print(abs(pos_end[0]-pos_start[0])>max_drift or abs(pos_end[1]-pos_start[1])>max_drift or 
       abs(pos_end[2]-pos_start[2])>max_drift or counterlogic.countdata.mean()<min_counts)
        return False
        write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_start[0], y_pos=pos_start[1], z_pos=pos_start[2],sucess=0)
    write_to_logfile('PositionRefocus_log', str(datetime.datetime.now()),name= setup['NV_name'], 
                     x_pos=pos_end[0], y_pos=pos_end[1], z_pos=pos_end[2],sucess=1)
    return True

def do_hahn_echo(generate_new=True, save_tag='',load_tag=''):
    # sanity check for long sequences:
    if T2['tau_start']+T2['tau_step']*T2['number_of_taus'] > 50e-3 :
        print("Waveform exceeds 3 ms too long for AWG7112")
        return True,0,0,0,0
    # generate ensemble object
    tau_arr = T2['tau_start']+T2['tau_step']*np.arange(T2['number_of_taus']) 
    tau_arr = 2*tau_arr # now it is full evolution time
    if generate_new:
        sequencegeneratorlogic.delete_ensemble('hahn_echo')
        sequencegeneratorlogic.generate_Hahn_echo('hahn_echo', T2['rabi_period'], 100.0e6, 0.25,
                      T2['tau_start'], T2['tau_step'], T2['number_of_taus'], 'a_ch1',
                      3.0e-6, 2.0, 0.7e-6, 1.0e-6, '', 'd_ch2')
        # sample, upload and load waveform
        if 'hahn_echo' not in sequencegeneratorlogic.saved_pulse_block_ensembles:
            sequencegeneratorlogic.log.error('Ensemble generation of Hahn echo in notebook timed out.')
            return
        pulsedmasterlogic.sample_block_ensemble('hahn_echo', True)
        
    else:
        pulsedmasterlogic.load_asset_into_channels('hahn_echo')
    while pulsedmasterlogic.status_dict['sauplo_ensemble_busy'] or pulsedmasterlogic.status_dict['loading_busy']:
        time.sleep(0.5)
    pulsedmasterlogic.do_fit('No Fit')
    
    # set parameters in analysis tab
    if T2['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*T2['number_of_taus'], 100e-6, [], T2['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, T2['number_of_taus'], 100e-6, [], T2['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < T2['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','T2'+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

def do_T1(save_tag='',load_tag=''):
    
    # generate sequence object
    tau_arr = T1['tau_start']+T1['tau_step']*np.arange(T1['number_of_taus'])
    pulsedmasterlogic.do_fit('No Fit')
    # set parameters in analysis tab
    if T1['alternating']:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, 2*T1['number_of_taus'], 100e-6, [], T1['alternating'])
    else:
        pulsedmasterlogic.measurement_sequence_settings_changed(tau_arr, T1['number_of_taus'], 100e-6, [], T1['alternating'])
    pulsedmasterlogic.fast_counter_settings_changed(setup['fc_binwidth'], setup['laser_length']+0.2e-6)
    pulsedmasterlogic.analysis_interval_changed(1)
    pulsedmeasurementlogic.measurement_tag = save_tag
    # perform measurement
    pulsedmasterlogic.start_measurement(load_tag)
    while not pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    user_terminated = False
    start_time = time.time()
    while time.time() - start_time < T1['measurement_time']:
        if not pulsedmasterlogic.status_dict['measurement_running']:
            user_terminated = True
            break
        time.sleep(0.5)
    pulsedmasterlogic.stop_measurement(save_tag)
    while pulsedmasterlogic.status_dict['measurement_running']:
        time.sleep(0.5)
    if save_tag=='':
        pulsedmasterlogic.save_measurement_data('s','T1'+'_'+setup['NV_name'],True)
    
    time.sleep(2)
    return user_terminated

In [11]:
# load saved T1 sequence
pulsedmasterlogic.load_asset_into_channels('T1 sequence')

## refocus methods

In [57]:
def do_hahn_echo_refocus(poi):
    end_measure=False
    T2_total_time = T2['measurement_time']
    T2_runtime = 0.0
    T2['measurement_time'] = T2['refocus_interval']
    end_measure = do_hahn_echo(True, 'hahn_echo_refocus','')
    T2_runtime += T2['refocus_interval']

    while T2_total_time > T2_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
#             pulsedmasterlogic.toggle_pulse_generator(True)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_hahn_echo(False, 'hahn_echo_refocus', 'hahn_echo_refocus')
        T2_runtime += T2['refocus_interval']
        if end_measure:
            break
    T2['measurement_time'] = T2_total_time
    # save measurement
    pulsedmasterlogic.save_measurement_data('s','T2'+'_'+setup['NV_name'],True)
    write_to_logfile('T2_log', str(datetime.datetime.now()),name= setup['NV_name'], runtime=T2_runtime)
    time.sleep(2)
    return end_measure

def do_T1_refocus(poi):
    end_measure=False
    T1_total_time = T1['measurement_time']
    T1_runtime = 0.0
    T1['measurement_time'] = T1['refocus_interval']
    end_measure = do_T1('T1_refocus','')
    T1_runtime += T1['refocus_interval']

    while T1_total_time > T1_runtime:
        for i in range(1):
            end_measure = not refocus_poi(poi)
#             pulsedmasterlogic.toggle_pulse_generator(True)
            if not end_measure:
                break
        if end_measure:
            break
        end_measure = do_T1('T1_refocus', 'T1_refocus')
        T1_runtime += T1['refocus_interval']
        if end_measure:
            break
    T1['measurement_time'] = T1_total_time
    # save measurement
    pulsedmasterlogic.save_measurement_data('s','T1'+'_'+setup['NV_name'],True)
    write_to_logfile('T1_log', str(datetime.datetime.now()),name= setup['NV_name'], runtime=T1_runtime)
    time.sleep(2)
    return end_measure

In [44]:
do_hahn_echo(False, 'hahn_echo_refocus', 'hahn_echo_refocus')

False

In [12]:
do_T1()

Traceback (most recent call last):
  File "C:\Users\pi3\Documents\GitHub\qudi\logic\jupyterkernel\qzmqkernel.py", line 795, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-d2073d982b51>", line 1, in <module>
    do_T1()
  File "<ipython-input-6-c4f6a43fb6e5>", line 120, in do_T1
    tau_arr = T1['tau_start']+T1['tau_step']*np.arange(T1['number_of_taus'])
NameError: name 'T1' is not defined


In [9]:
type(T1['rabi_period'])

<class 'float'>